In [125]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
from time import sleep
# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv()



True

In [126]:
# Invocamos de forma segura la contraseña de rapidapi
key = os.getenv("token")

In [127]:
url = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {"query":"Barcelona", "limit":"50", "adults": "2"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

respuesta=response.json()

In [128]:
with open('hoteleles_barcelona.json', 'w') as archivo:
    json.dump(respuesta, archivo, indent=4)

In [129]:
respuesta

{'link': 'https://www.tripadvisor.com/Hotels-g187497-Barcelona_Catalonia-Hotels.html',
 'total_pages': 73,
 'current_page': 1,
 'total_items_count': 2185,
 'items_count': 31,
 'results': [{'id': 2545026,
   'name': 'Andante Hotel',
   'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html',
   'reviews': 2817,
   'rating': 4.5,
   'price_range_usd': {'min': 150, 'max': 241},
   'phone': '+34 934 41 25 45',
   'address': 'Av. Drassanes, 23-25 Rambla Del Raval, 08001 Barcelona Spain',
   'detailed_address': {'street': 'Av. Drassanes, 23-25, Rambla Del Raval',
    'city': 'Barcelona',
    'postal_code': '08001',
    'state': None,
    'country': 'Spain'},
   'ranking': {'current_rank': 91, 'total': 512},
   'featured_image': None,
   'latitude': 41.37648,
   'longitude': 2.172685,
   'amenities': ['Free Wifi', 'Pool', 'Outdoor pool', 'Fitness center'],
   'providers': ['Expedia.com', 'Booking.com', 'Hotels.com', 'ZenHotels.com']},

In [130]:
url = "https://tripadvisor-scraper.p.rapidapi.com/hotels/list"

querystring = {"query":"Santiago de Compostela", "limit":"50", "adults": "2"}

headers = {
	"x-rapidapi-key": key,
	"x-rapidapi-host": "tripadvisor-scraper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

respuesta2=response.json()

In [131]:
respuesta2

{'link': 'https://www.tripadvisor.com/Hotels-g187508-Santiago_de_Compostela_Province_of_A_Coruna_Galicia-Hotels.html',
 'total_pages': 11,
 'current_page': 1,
 'total_items_count': 307,
 'items_count': 34,
 'results': [{'id': 1510515,
   'name': 'Hotel Spa Relais & Chateaux A Quinta da Auga',
   'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d1510515-Reviews-Hotel_Spa_Relais_Chateaux_A_Quinta_da_Auga-Santiago_de_Compostela_Province_of_A_Coruna.html',
   'reviews': 1116,
   'rating': 4.5,
   'price_range_usd': {'min': 207, 'max': 297},
   'phone': '+34 981 53 46 36',
   'address': 'Paseo Amaia 23B Urbanización Brandia. Vidán, 15706, Santiago de Compostela Spain',
   'detailed_address': {'street': 'Paseo Amaia 23B, Urbanización Brandia. Vidán',
    'city': None,
    'postal_code': '15706',
    'state': None,
    'country': 'Spain'},
   'ranking': {'current_rank': 8, 'total': 69},
   'featured_image': None,
   'latitude': 42.870335,
   'longitude': -8.582583,
   'amenities': ['

In [132]:
with open('hoteleles_santiago.json', 'w') as archivo:
    json.dump(respuesta2, archivo, indent=4)

In [133]:
def extraer_informacion_hoteles(respuesta_json):
    """
    Extrae información relevante sobre hoteles a partir de una respuesta JSON de un servicio de búsqueda de hoteles.

    La función itera sobre los resultados del JSON y extrae información clave sobre cada hotel, 
    incluyendo el nombre, número de reseñas, rating, rango de precios, ciudad, servicios ofrecidos y un enlace al hotel.
    Calcula además el precio medio por noche basado en el rango de precios.

    Args:
        respuesta_json (dict): Un diccionario que representa la respuesta en formato JSON de un servicio de búsqueda de hoteles.

    Returns:
        list[dict]: Una lista de diccionarios, donde cada diccionario contiene la información de un hotel con las siguientes claves:
            - nombre: El nombre del hotel.
            - numero_reviews: Número de reseñas del hotel.
            - rating: Rating del hotel.
            - precio_rango_min: El precio mínimo por noche en USD.
            - precio_rango_max: El precio máximo por noche en USD.
            - precio_medio_por noche: Precio medio por noche calculado a partir del rango de precios.
            - ciudad: Ciudad donde se encuentra el hotel.
            - servicios: Lista de servicios que ofrece el hotel.
            - link: URL que enlaza a la página del hotel.
    """
    lista_diccionarios = []
    res = respuesta_json
    for i in range(0, len(res.get("results", []))):

        nombre = res["results"][i].get("name", "N/A")
        numero_reviews = res["results"][i].get("reviews", 0)
        rating = res["results"][i].get("rating", "N/A")
        
        rango_precios = res["results"][i].get("price_range_usd", {})
        precio_rango_min = rango_precios.get("min", 0)
        precio_rango_max = rango_precios.get("max", 0)
        precio_medio = (precio_rango_max + precio_rango_min) / 2 if (precio_rango_max + precio_rango_min) > 0 else 0
        
        ciudad = res["results"][i].get("detailed_address", {}).get("city", "N/A")
        servicios = res["results"][i].get("amenities", [])
        link = res["results"][i].get("link", "N/A")

        diccionario_hotel= {"nombre":nombre,
                "numero_reviews":numero_reviews,
                "rating": rating,
                "precio_rango_min":precio_rango_min,
                "precio_rango_max":precio_rango_max,
                "precio_medio_por noche":precio_medio,
                "ciudad":ciudad,
                "servicios":servicios,
                "link":link
                }
        lista_diccionarios.append(diccionario_hotel)
    return lista_diccionarios

In [134]:
extraer_informacion_hoteles(respuesta)

[{'nombre': 'Andante Hotel',
  'numero_reviews': 2817,
  'rating': 4.5,
  'precio_rango_min': 150,
  'precio_rango_max': 241,
  'precio_medio_por noche': 195.5,
  'ciudad': 'Barcelona',
  'servicios': ['Free Wifi', 'Pool', 'Outdoor pool', 'Fitness center'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html'},
 {'nombre': 'Hyatt Regency Barcelona Tower',
  'numero_reviews': 3566,
  'rating': 4.5,
  'precio_rango_min': 187,
  'precio_rango_max': 295,
  'precio_medio_por noche': 241.0,
  'ciudad': 'Barcelona',
  'servicios': ['Free Wifi',
   'Pool',
   'Heated pool',
   'Indoor pool',
   'Restaurant'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187497-d630596-Reviews-Hyatt_Regency_Barcelona_Tower-Barcelona_Catalonia.html'},
 {'nombre': 'Hotel Astoria',
  'numero_reviews': 1900,
  'rating': 4.5,
  'precio_rango_min': 118,
  'precio_rango_max': 233,
  'precio_medio_por noche': 175.5,
  'ciudad': 'Barcelona',
  'serv

In [135]:
extraer_informacion_hoteles(respuesta2)

[{'nombre': 'Hotel Spa Relais & Chateaux A Quinta da Auga',
  'numero_reviews': 1116,
  'rating': 4.5,
  'precio_rango_min': 207,
  'precio_rango_max': 297,
  'precio_medio_por noche': 252.0,
  'ciudad': None,
  'servicios': ['Free Wifi',
   'Free parking',
   'Breakfast included',
   'Heated pool',
   'Indoor pool'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d1510515-Reviews-Hotel_Spa_Relais_Chateaux_A_Quinta_da_Auga-Santiago_de_Compostela_Province_of_A_Coruna.html'},
 {'nombre': 'NH Collection Santiago de Compostela',
  'numero_reviews': 2316,
  'rating': 4.5,
  'precio_rango_min': 97,
  'precio_rango_max': 177,
  'precio_medio_por noche': 137.0,
  'ciudad': None,
  'servicios': ['Free Wifi',
   'Free parking',
   'Pool',
   'Heated pool',
   'Indoor pool'],
  'link': 'https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html'},
 {'nombre': 'Hotel Compostela',
  'nume

In [136]:
def crear_df_hoteles(funcion_extraer_informacion_hoteles):
    """
    Crea y devuelve un DataFrame a partir de la información de hoteles obtenida.

    La función recibe una lista de diccionarios generada por la función `extraer_informacion_hoteles`, 
    convierte esa lista en un DataFrame y elimina las columnas "precio_rango_min" y "precio_rango_max",
    manteniendo solo la información relevante para el análisis.

    Args:
        funcion_extraer_informacion_hoteles (list[dict]): Lista de diccionarios con información sobre hoteles, 
        obtenida de la función `extraer_informacion_hoteles`.

    Returns:
        pd.DataFrame: Un DataFrame con los datos de hoteles, excluyendo las columnas "precio_rango_min" y "precio_rango_max".
    """

    df_hotel= pd.DataFrame(funcion_extraer_informacion_hoteles).drop(columns= ["precio_rango_min", "precio_rango_max"])
    pd.set_option('display.max_colwidth', None)

    df_hotel
    return df_hotel

In [137]:
df_hoteles_barcelona=crear_df_hoteles(extraer_informacion_hoteles(respuesta))
df_hoteles_barcelona.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,Andante Hotel,2817,4.5,195.5,Barcelona,"[Free Wifi, Pool, Outdoor pool, Fitness center]",https://www.tripadvisor.com/Hotel_Review-g187497-d2545026-Reviews-Andante_Hotel-Barcelona_Catalonia.html
1,Hyatt Regency Barcelona Tower,3566,4.5,241.0,Barcelona,"[Free Wifi, Pool, Heated pool, Indoor pool, Restaurant]",https://www.tripadvisor.com/Hotel_Review-g187497-d630596-Reviews-Hyatt_Regency_Barcelona_Tower-Barcelona_Catalonia.html
2,Hotel Astoria,1900,4.5,175.5,Barcelona,"[Free Wifi, Pool, Outdoor pool, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187497-d228590-Reviews-Hotel_Astoria-Barcelona_Catalonia.html
3,Majestic Hotel & Spa Barcelona,4259,4.5,446.0,Barcelona,"[Free Wifi, Pool, Outdoor pool, Restaurant, Airport transportation]",https://www.tripadvisor.com/Hotel_Review-g187497-d190616-Reviews-Majestic_Hotel_Spa_Barcelona-Barcelona_Catalonia.html
4,Hotel Catalonia Park Güell,1417,3.5,135.5,Barcelona,"[Free Wifi, Outdoor pool, Restaurant, Room service, Pets Allowed]",https://www.tripadvisor.com/Hotel_Review-g187497-d236194-Reviews-Hotel_Catalonia_Park_Guell-Barcelona_Catalonia.html


In [138]:
df_hoteles_santiago=crear_df_hoteles(extraer_informacion_hoteles(respuesta2))
df_hoteles_santiago.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,Hotel Spa Relais & Chateaux A Quinta da Auga,1116,4.5,252.0,None,"[Free Wifi, Free parking, Breakfast included, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d1510515-Reviews-Hotel_Spa_Relais_Chateaux_A_Quinta_da_Auga-Santiago_de_Compostela_Province_of_A_Coruna.html
1,NH Collection Santiago de Compostela,2316,4.5,137.0,None,"[Free Wifi, Free parking, Pool, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html
2,Hotel Compostela,764,4.0,124.0,None,"[Free Wifi, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d248531-Reviews-Hotel_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
3,Hotel Restaurante Asador De Roxos Casa Albardonedo,233,4.5,91.0,Roxos,"[Free Wifi, Free parking, Breakfast included, Restaurant, Pets Allowed]",https://www.tripadvisor.com/Hotel_Review-g1870402-d1836140-Reviews-Hotel_Restaurante_Asador_De_Roxos_Casa_Albardonedo-Roxos_Santiago_de_Compostela_Provi.html
4,Gran Hotel Los Abetos,808,4.5,135.5,None,"[Free Wifi, Free parking, Outdoor pool, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187508-d231702-Reviews-Gran_Hotel_Los_Abetos-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html


Veamos si es necesario limpiar los dataframes que acabamos de obtener

In [139]:
df_hoteles_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre                  31 non-null     object 
 1   numero_reviews          31 non-null     int64  
 2   rating                  31 non-null     float64
 3   precio_medio_por noche  31 non-null     float64
 4   ciudad                  31 non-null     object 
 5   servicios               31 non-null     object 
 6   link                    31 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.8+ KB


In [140]:
df_hoteles_santiago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre                  34 non-null     object 
 1   numero_reviews          34 non-null     int64  
 2   rating                  34 non-null     float64
 3   precio_medio_por noche  34 non-null     float64
 4   ciudad                  1 non-null      object 
 5   servicios               34 non-null     object 
 6   link                    34 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 2.0+ KB


Observamos que los tipos de datos son coherentes y manejables para el análisis en ambos dataframes. En cuanto a valores nulos, el dataframe de barcelona, no tiene ninguno, mientras que el de santiago los acumula en la columna ciudad. Esto es fácilmente solucionable, rellenando nosotros esos valores pues sabems que la ciudad es Santiago de Compostela.

In [141]:
df_hoteles_santiago['ciudad'] = 'Santiago de Compostela'
df_hoteles_santiago.head()

,nombre,numero_reviews,rating,precio_medio_por noche,ciudad,servicios,link
0,Hotel Spa Relais & Chateaux A Quinta da Auga,1116,4.5,252.0,Santiago de Compostela,"[Free Wifi, Free parking, Breakfast included, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d1510515-Reviews-Hotel_Spa_Relais_Chateaux_A_Quinta_da_Auga-Santiago_de_Compostela_Province_of_A_Coruna.html
1,NH Collection Santiago de Compostela,2316,4.5,137.0,Santiago de Compostela,"[Free Wifi, Free parking, Pool, Heated pool, Indoor pool]",https://www.tripadvisor.com/Hotel_Review-g187508-d584984-Reviews-NH_Collection_Santiago_de_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galici.html
2,Hotel Compostela,764,4.0,124.0,Santiago de Compostela,"[Free Wifi, Room service, Bar/Lounge]",https://www.tripadvisor.com/Hotel_Review-g187508-d248531-Reviews-Hotel_Compostela-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html
3,Hotel Restaurante Asador De Roxos Casa Albardonedo,233,4.5,91.0,Santiago de Compostela,"[Free Wifi, Free parking, Breakfast included, Restaurant, Pets Allowed]",https://www.tripadvisor.com/Hotel_Review-g1870402-d1836140-Reviews-Hotel_Restaurante_Asador_De_Roxos_Casa_Albardonedo-Roxos_Santiago_de_Compostela_Provi.html
4,Gran Hotel Los Abetos,808,4.5,135.5,Santiago de Compostela,"[Free Wifi, Free parking, Outdoor pool, Restaurant, Room service]",https://www.tripadvisor.com/Hotel_Review-g187508-d231702-Reviews-Gran_Hotel_Los_Abetos-Santiago_de_Compostela_Province_of_A_Coruna_Galicia.html


Comprobemos por último si hay filas duplicadas y en caso de haberlas eliminamos el duplicado

In [142]:
#Excluimos de la comprobación la columna servicios, que da problemas por contener lsitas en cada fila
num_filas_duplicadas = df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts()
num_filas_duplicadas

False    30
True      1
Name: count, dtype: int64

In [143]:
# Para eliminar la fila duplicada, no nos vale aplicar directamente drop_duplicates
# por ser "servicios" una columna de listas

df_auxiliar = df_hoteles_barcelona.drop(columns=['servicios'])


filas_no_duplicadas = ~df_auxiliar.duplicated()

# Obtenemos el dataFrame original filtrado para eliminar las filas duplicadas
df_hoteles_barcelona = df_hoteles_barcelona[filas_no_duplicadas].reset_index(drop=True)

In [144]:
df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts() #vemos que ya no hay duplicados

False    30
Name: count, dtype: int64

In [145]:
# Mismo proceso para el DataFrame de Santiago de Compostela
num_filas_duplicadas = df_hoteles_barcelona.drop(columns=['servicios']).duplicated().value_counts()
num_filas_duplicadas

False    30
Name: count, dtype: int64

En este caso, el de Santiago no tiene duplicados, por lo que termina aquí la limpieza de ambos DataFrames

**Vuelos con Skyscraper**

In [146]:

url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {
    "originSkyId": "MAD",  # Código para Madrid
    "destinationSkyId": "SCQ",  # Código para Santiago de Compostela
    "originEntityId": "95565077",  # Id de Madrid
    "destinationEntityId": "95565108",  # Id de Santiago de Compostela
    "date": "2024-11-22",
    "returnDate":"2024-11-24",
    "cabinClass": "economy",
    "adults": "2",
    "sortBy": "best",
    "limit":50,
    "currency": "EUR",
}

headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

vuelo_santiago = response.json()


In [147]:
vuelo_santiago

{'status': True,
 'timestamp': 1729452287266,
 'sessionId': 'd14a4a6c-a611-44ef-ac04-8c19d23af4c2',
 'data': {'context': {'status': 'incomplete',
   'sessionId': 'ClQIARJQCk4KJGQxNGE0YTZjLWE2MTEtNDRlZi1hYzA0LThjMTlkMjNhZjRjMhACGiQyNjIxZmIzNi1mM2U1LTRhMjYtYmQ5YS02OTQxODUyYTU1YzQSKHVzc181N2ViNWY1Ni01ZDE3LTRmN2YtYTNjMS0wMjQwNTE5ZmRkNzk=',
   'totalResults': 10},
  'itineraries': [{'id': '13870-2411221055--31915-0-16142-2411221215|16142-2411240805--31915-0-13870-2411240920',
    'price': {'raw': 88.9,
     'formatted': '89 €',
     'pricingOptionId': 'o8tWlBOBv3Dh'},
    'legs': [{'id': '13870-2411221055--31915-0-16142-2411221215',
      'origin': {'id': 'MAD',
       'entityId': '95565077',
       'name': 'Madrid',
       'displayCode': 'MAD',
       'city': 'Madrid',
       'country': 'Spain',
       'isHighlighted': False},
      'destination': {'id': 'SCQ',
       'entityId': '95565108',
       'name': 'Santiago de Compostela',
       'displayCode': 'SCQ',
       'city': 'Santiago de C

In [148]:
with open('santiago.json', 'w') as archivo:
     json.dump(vuelo_santiago, archivo, indent=4)

In [149]:

url = "https://sky-scrapper.p.rapidapi.com/api/v2/flights/searchFlights"

querystring = {
    "originSkyId": "MAD",  # Código para Madrid
    "destinationSkyId": "BCN",  # Código para Barcelona
    "originEntityId": "95565077",  # Id de Madrid
    "destinationEntityId": "95565085",  # Id de Barcelona
    "date": "2024-11-22",
    "returnDate":"2024-11-24",
    "cabinClass": "economy",
    "adults": "2",
    "sortBy": "best",
    "limit":50,
    "currency": "EUR",
}

headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "sky-scrapper.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

vuelo_barcelona = response.json()


In [150]:
vuelo_barcelona

{'status': True,
 'timestamp': 1729452289840,
 'sessionId': 'aaf80f2f-78f0-4fc9-a5c2-40ccbeecad63',
 'data': {'context': {'status': 'incomplete',
   'sessionId': 'ClQIARJQCk4KJGFhZjgwZjJmLTc4ZjAtNGZjOS1hNWMyLTQwY2NiZWVjYWQ2MxACGiRiYThiNzdlNi03YjE5LTRhMmItYTIwMC03YjM3MDcwYTQzYWISKHVzc19kOGJjMzI5NC1iOWM1LTQxMmEtODAwNy05YTY4MGY5YmViODM=',
   'totalResults': 10},
  'itineraries': [{'id': '13870-2411220715--32222-0-9772-2411220830|9772-2411241515--32222-0-13870-2411241640',
    'price': {'raw': 117.98,
     'formatted': '118 €',
     'pricingOptionId': 'xg7VgqueIUeJ'},
    'legs': [{'id': '13870-2411220715--32222-0-9772-2411220830',
      'origin': {'id': 'MAD',
       'entityId': '95565077',
       'name': 'Madrid',
       'displayCode': 'MAD',
       'city': 'Madrid',
       'country': 'Spain',
       'isHighlighted': False},
      'destination': {'id': 'BCN',
       'entityId': '95565085',
       'name': 'Barcelona',
       'displayCode': 'BCN',
       'city': 'Barcelona',
       'countr

In [151]:
with open('barcelona.json', 'w') as archivo:
     json.dump(vuelo_barcelona, archivo, indent=4)

In [152]:
def extraer_informacion_vuelos(respuesta_json):
    """
    Extrae información relevante sobre vuelos a partir de una respuesta JSON de un servicio de itinerarios de vuelos.

    La función itera sobre los vuelos y sus tramos (piernas) en el JSON de respuesta, extrayendo información
    como el precio, aeropuertos de origen y destino, tiempos de salida y llegada, duración del vuelo, escalas y aerolíneas.
    Luego organiza esta información en una lista de diccionarios.

    Args:
        respuesta_json (dict): Un diccionario que representa la respuesta en formato JSON de un servicio de vuelos.

    Returns:
        list[dict]: Una lista de diccionarios, donde cada diccionario contiene la información de un tramo de vuelo con las siguientes claves:
            - id_origen: ID del aeropuerto de origen.
            - ciudad_origen: Nombre de la ciudad de origen.
            - id_destino: ID del aeropuerto de destino.
            - ciudad_destino: Nombre de la ciudad de destino.
            - salida: Hora y fecha de salida del vuelo.
            - llegada: Hora y fecha de llegada del vuelo.
            - duracion(min): Duración del vuelo en minutos.
            - escalas: Número de escalas del vuelo.
            - aerolinea: Nombre de la aerolínea que opera el vuelo.
            - precio: Precio del vuelo (formato raw).
    """
    lista_diccionarios = []
    res = respuesta_json.get("data", {})

    for vuelo in range(0, len(res.get("itineraries", []))): 
        precio = res["itineraries"][vuelo].get("price", {}).get("raw", 0)

        for i in range(0, 2):
            id_origen = res["itineraries"][vuelo]["legs"][i].get("origin", {}).get("id", "N/A")
            origen = res["itineraries"][vuelo]["legs"][i].get("origin", {}).get("city", "N/A")

            id_destino = res["itineraries"][vuelo]["legs"][i].get("destination", {}).get("id", "N/A")
            destino = res["itineraries"][vuelo]["legs"][i].get("destination", {}).get("city", "N/A")

            duracion_minutos = res["itineraries"][vuelo]["legs"][i].get("durationInMinutes", 0)
            escalas = res["itineraries"][vuelo]["legs"][i].get("stopCount", 0)

            salida = res["itineraries"][vuelo]["legs"][i].get("departure", "N/A")
            llegada = res["itineraries"][vuelo]["legs"][i].get("arrival", "N/A")

            aerolinea = res["itineraries"][vuelo]["legs"][i].get("carriers", {}).get("marketing", [{}])[0].get("name", "N/A")

            diccionario_vuelos = {
                "id_origen": id_origen,
                "ciudad_origen": origen,
                "id_destino": id_destino,
                "ciudad_destino": destino,
                "salida": salida,
                "llegada": llegada,
                "duracion(min)": duracion_minutos,
                "escalas": escalas,
                "aerolinea": aerolinea,
                "precio": precio
            }
            lista_diccionarios.append(diccionario_vuelos)
    
    return lista_diccionarios
 

In [153]:
extraer_informacion_vuelos(vuelo_santiago)

[{'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'SCQ',
  'ciudad_destino': 'Santiago de Compostela',
  'salida': '2024-11-22T10:55:00',
  'llegada': '2024-11-22T12:15:00',
  'duracion(min)': 80,
  'escalas': 0,
  'aerolinea': 'Ryanair',
  'precio': 88.9},
 {'id_origen': 'SCQ',
  'ciudad_origen': 'Santiago de Compostela',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T08:05:00',
  'llegada': '2024-11-24T09:20:00',
  'duracion(min)': 75,
  'escalas': 0,
  'aerolinea': 'Ryanair',
  'precio': 88.9},
 {'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'SCQ',
  'ciudad_destino': 'Santiago de Compostela',
  'salida': '2024-11-22T11:50:00',
  'llegada': '2024-11-22T13:05:00',
  'duracion(min)': 75,
  'escalas': 0,
  'aerolinea': 'Iberia',
  'precio': 178.94},
 {'id_origen': 'SCQ',
  'ciudad_origen': 'Santiago de Compostela',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T13:45:00',
  'llegada': '2024-

In [154]:
extraer_informacion_vuelos(vuelo_barcelona)

[{'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'BCN',
  'ciudad_destino': 'Barcelona',
  'salida': '2024-11-22T07:15:00',
  'llegada': '2024-11-22T08:30:00',
  'duracion(min)': 75,
  'escalas': 0,
  'aerolinea': 'Iberia',
  'precio': 117.98},
 {'id_origen': 'BCN',
  'ciudad_origen': 'Barcelona',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T15:15:00',
  'llegada': '2024-11-24T16:40:00',
  'duracion(min)': 85,
  'escalas': 0,
  'aerolinea': 'Iberia',
  'precio': 117.98},
 {'id_origen': 'MAD',
  'ciudad_origen': 'Madrid',
  'id_destino': 'BCN',
  'ciudad_destino': 'Barcelona',
  'salida': '2024-11-22T10:30:00',
  'llegada': '2024-11-22T11:45:00',
  'duracion(min)': 75,
  'escalas': 0,
  'aerolinea': 'Iberia',
  'precio': 149.48},
 {'id_origen': 'BCN',
  'ciudad_origen': 'Barcelona',
  'id_destino': 'MAD',
  'ciudad_destino': 'Madrid',
  'salida': '2024-11-24T15:15:00',
  'llegada': '2024-11-24T16:40:00',
  'duracion(min)': 85,
  'escalas

In [155]:
def crear_df_vuelos(funcion_informacion_vuelos):
    """
    Crea y devuelve un DataFrame a partir de la información de vuelos obtenida.

    La función recibe una lista de diccionarios generada por la función `extraer_informacion_vuelos`, 
    convierte esa lista en un DataFrame y formatea las columnas de "salida" y "llegada" 
    como objetos datetime.

    Args:
        funcion_informacion_vuelos (list[dict]): Lista de diccionarios con información sobre vuelos, 
        obtenida de la función `extraer_informacion_vuelos`.

    Returns:
        pd.DataFrame: Un DataFrame con los datos de vuelos, donde las columnas "salida" y "llegada"
        están formateadas como datetime.
    """

    df_vuelos= pd.DataFrame(funcion_informacion_vuelos)
    df_vuelos["salida"]= pd.to_datetime(df_vuelos["salida"])
    df_vuelos["llegada"]= pd.to_datetime(df_vuelos["llegada"])
    return df_vuelos

In [156]:
santiago=extraer_informacion_vuelos(vuelo_santiago)
df_vuelos_santiago=crear_df_vuelos(santiago)

In [157]:
barcelona=extraer_informacion_vuelos(vuelo_barcelona)
df_vuelos_barcelona=crear_df_vuelos(barcelona)

In [158]:
df_vuelos_santiago.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 10:55:00,2024-11-22 12:15:00,80,0,Ryanair,88.90
1,SCQ,Santiago de Compostela,MAD,Madrid,2024-11-24 08:05:00,2024-11-24 09:20:00,75,0,Ryanair,88.90
2,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 11:50:00,2024-11-22 13:05:00,75,0,Iberia,178.94
3,SCQ,Santiago de Compostela,MAD,Madrid,2024-11-24 13:45:00,2024-11-24 15:00:00,75,0,Iberia,178.94
4,MAD,Madrid,SCQ,Santiago de Compostela,2024-11-22 07:30:00,2024-11-22 08:45:00,75,0,Iberia,134.00


In [159]:
df_vuelos_barcelona.head()

,id_origen,ciudad_origen,id_destino,ciudad_destino,salida,llegada,duracion(min),escalas,aerolinea,precio
0,MAD,Madrid,BCN,Barcelona,2024-11-22 07:15:00,2024-11-22 08:30:00,75,0,Iberia,117.98
1,BCN,Barcelona,MAD,Madrid,2024-11-24 15:15:00,2024-11-24 16:40:00,85,0,Iberia,117.98
2,MAD,Madrid,BCN,Barcelona,2024-11-22 10:30:00,2024-11-22 11:45:00,75,0,Iberia,149.48
3,BCN,Barcelona,MAD,Madrid,2024-11-24 15:15:00,2024-11-24 16:40:00,85,0,Iberia,149.48
4,MAD,Madrid,BCN,Barcelona,2024-11-22 06:45:00,2024-11-22 08:00:00,75,0,Iberia,111.98


Veamos si es necesario limpiar los archivos

In [160]:
df_vuelos_santiago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


In [161]:
df_vuelos_barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_origen       20 non-null     object        
 1   ciudad_origen   20 non-null     object        
 2   id_destino      20 non-null     object        
 3   ciudad_destino  20 non-null     object        
 4   salida          20 non-null     datetime64[ns]
 5   llegada         20 non-null     datetime64[ns]
 6   duracion(min)   20 non-null     int64         
 7   escalas         20 non-null     int64         
 8   aerolinea       20 non-null     object        
 9   precio          20 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(5)
memory usage: 1.7+ KB


No hay valores nulos que tratar, veamos que pasa con los duplicados.

In [162]:
df_vuelos_santiago.duplicated().value_counts()

False    20
Name: count, dtype: int64

In [163]:
df_vuelos_barcelona.duplicated().value_counts()

False    19
True      1
Name: count, dtype: int64

In [164]:
# Tenemos una fila duplicada en cada Data Frame, las eliminamos
df_vuelos_santiago=df_vuelos_santiago.drop_duplicates()
df_vuelos_barcelona=df_vuelos_barcelona.drop_duplicates()